<a href="https://colab.research.google.com/github/psasanka1729/gpu_programs/blob/main/generate_H_1_0_H_0_1_L_k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import numpy as np
# import matplotlib.pyplot as plt
from k_local_hamiltonian import KLocalHamiltonian

In [ ]:
adjacent = True

L = 15
k = 7

number_of_pauli_strings = (L - k + 1) * 3**k if adjacent else math.comb(L, k) * 3**k

random_coefficients_0_1 = np.random.normal(0, 1, number_of_pauli_strings)
# np.save(f'random_coefficients_L_{L}_k_{k}_sigma_1_short_ranged.npy', random_coefficients_0_1)

hamiltonian_generator = KLocalHamiltonian(L=L, k=k, random_coefficients=random_coefficients_0_1, adjacent=adjacent)

pauli_strings = hamiltonian_generator.generate_k_local_pauli_strings()

In [ ]:
# import numpy as np
# import math
# import os
# import multiprocessing
# import time
# from scipy.sparse import save_npz
# from k_local_hamiltonian import KLocalHamiltonian

# # --- Global variables accessible by worker processes ---
# # These are defined globally so we don't need to pass them to each worker,
# # which is more efficient.
# hamiltonian_generator = None
# OUTPUT_DIR = "pauli_matrices"

# def process_and_save(args):
#     """
#     Worker function that computes one matrix and saves it to a unique .npz file.
#     This function will be executed by each core in parallel.

#     Args:
#         args (tuple): A tuple containing the item's index and its Pauli string.
#     """
#     index, pauli_string = args

#     # 1. Compute the matrix for the given Pauli string
#     matrix = hamiltonian_generator.tensor_product(pauli_string)

#     # 2. Define a unique filename using the index
#     filename = os.path.join(OUTPUT_DIR, f"{pauli_string}.npz")

#     # 3. Save the sparse matrix to the file
#     save_npz(filename, matrix)

#     # Return the filename for logging or tracking progress
#     return filename

# if __name__ == "__main__":
#     adjacent = True
#     L = 15
#     k = 7 # WARNING: Real L=15, k=7 values will generate millions of files and use immense disk space.

#     # --- Setup ---
#     number_of_pauli_strings = (L - k + 1) * 3**k if adjacent else math.comb(L, k) * 3**k
#     random_coefficients = np.random.normal(0, 1, number_of_pauli_strings)

#     # Initialize the generator object. It will be inherited by the worker processes.
#     hamiltonian_generator = KLocalHamiltonian(L=L, k=k, random_coefficients=random_coefficients, adjacent=adjacent)
#     pauli_strings = hamiltonian_generator.generate_k_local_pauli_strings()

#     # --- Parallel Execution ---

#     # Create the output directory if it doesn't already exist
#     os.makedirs(OUTPUT_DIR, exist_ok=True)
#     print(f"Matrices will be saved in the '{OUTPUT_DIR}/' directory.")

#     # Automatically get the number of available CPU cores
#     n_cores = os.cpu_count()
#     print(f"Detected {n_cores} cores. Starting parallel file saving.")

#     # Prepare arguments for the parallel map: a list of (index, pauli_string) tuples
#     work_items = enumerate(pauli_strings)

#     # Create a pool of workers and distribute the file-saving task
#     with multiprocessing.Pool(processes=n_cores) as pool:
#         # The map function applies 'process_and_save' to each item in 'work_items'
#         saved_files = pool.map(process_and_save, work_items)

#     print(f"\nProcessing complete. Saved {len(saved_files)} matrix files.")

Matrices will be saved in the 'pauli_matrices/' directory.
Detected 8 cores. Starting parallel file saving.


Process ForkPoolWorker-82:
Process ForkPoolWorker-79:
Process ForkPoolWorker-80:
Process ForkPoolWorker-78:
Process ForkPoolWorker-81:


KeyboardInterrupt: 

In [ ]:
# # 1. Uninstall all existing CuPy versions to resolve the conflict
# !pip uninstall -y cupy-cuda11x cupy-cuda12x

# # 2. Install the correct CuPy version for Colab's current CUDA toolkit (12.x)
# !pip install cupy-cuda12x

Found existing installation: cupy-cuda11x 13.5.1
Uninstalling cupy-cuda11x-13.5.1:
  Successfully uninstalled cupy-cuda11x-13.5.1
Found existing installation: cupy-cuda12x 13.3.0
Uninstalling cupy-cuda12x-13.3.0:
  Successfully uninstalled cupy-cuda12x-13.3.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.2/113.2 MB 21.8 MB/s eta 0:00:00


In [ ]:
# If you haven't already, ensure the correct CuPy version is installed
# !pip uninstall -y cupy-cuda11x cupy-cuda12x
# !pip install cupy-cuda12x
# Then restart the session: Runtime -> Restart session

import cupy as cp
import cupyx.scipy.sparse as cupy_sparse
import numpy as np
import math
import os
import time
import itertools
from scipy.sparse import save_npz

class GPUKLocalHamiltonian:
    """
    A GPU-aware class. This version now systematically generates all
    k-local Pauli strings instead of a random subset.
    """
    def __init__(self, L, k, random_coefficients, adjacent):
        self.L = L
        self.k = k
        self.adjacent = adjacent
        self.coeffs = cp.asarray(random_coefficients)

        # Define Pauli matrices directly on the GPU
        self.pauli_map = {
            'I': cupy_sparse.identity(2, format='csc', dtype=cp.complex128),
            'X': cupy_sparse.csc_matrix(cp.array([[0, 1], [1, 0]], dtype=cp.complex128)),
            'Y': cupy_sparse.csc_matrix(cp.array([[0, -1j], [1j, 0]], dtype=cp.complex128)),
            'Z': cupy_sparse.csc_matrix(cp.array([[1, 0], [0, -1]], dtype=cp.complex128))
        }

    def generate_k_local_pauli_strings(self):
        """
        Systematically generates all k-local Pauli strings of length L.
        A k-local string has k non-identity operators.
        """
        generated_strings = []
        pauli_chars_non_identity = ['X', 'Y', 'Z']

        # Define the set of positions for the non-identity operators
        if self.adjacent:
            # Positions are contiguous blocks of size k
            position_sets = [tuple(range(i, i + self.k)) for i in range(self.L - self.k + 1)]
        else:
            # Positions can be any combination of k indices
            position_sets = list(itertools.combinations(range(self.L), self.k))

        # Generate all possible strings of 'X', 'Y', 'Z' of length k
        k_strings = list(itertools.product(pauli_chars_non_identity, repeat=self.k))

        # Combine the positions and the k-strings to create the full Pauli strings
        for positions in position_sets:
            for k_str in k_strings:
                final_string_list = ['I'] * self.L
                for i, pos in enumerate(positions):
                    final_string_list[pos] = k_str[i]
                generated_strings.append("".join(final_string_list))

        return generated_strings

    def tensor_product(self, pauli_string):
        """Computes the tensor product on the GPU based on the input string."""
        matrices_to_multiply = [self.pauli_map[char] for char in pauli_string]
        final_matrix = matrices_to_multiply[0]
        for i in range(1, len(matrices_to_multiply)):
            final_matrix = cupy_sparse.kron(final_matrix, matrices_to_multiply[i])
        return final_matrix

if __name__ == "__main__":
    # --- GPU Availability Check ---
    try:
        device_id = cp.cuda.runtime.getDevice()
        print(f"GPU Device {device_id} detected. Running on GPU.")
    except cp.cuda.runtime.CUDARuntimeError as e:
        print(f"GPU not detected or CuPy installation issue. Error: {e}")
        exit()

    # --- Setup ---
    adjacent = True
    # NOTE: Using smaller L and k for a quick demonstration.
    # You can set these back to L=15, k=7 for your full computation.
    L = 15
    k = 7
    OUTPUT_DIR = "gpu_pauli_matrices"

    os.makedirs(OUTPUT_DIR, exist_ok=True)
    print(f"Matrices will be saved in the '{OUTPUT_DIR}/' directory.")

    number_of_pauli_strings = (L - k + 1) * (3**k) if adjacent else math.comb(L, k) * (3**k)
    random_coefficients_cpu = np.random.normal(0, 1, number_of_pauli_strings)

    gpu_hamiltonian_generator = GPUKLocalHamiltonian(L=L, k=k, random_coefficients=random_coefficients_cpu, adjacent=adjacent)
    pauli_strings = gpu_hamiltonian_generator.generate_k_local_pauli_strings()

    # --- SAVE THE GENERATED PAULI STRINGS TO A FILE ---
    strings_filename = "pauli_strings.txt"
    with open(strings_filename, 'w') as f:
        for string in pauli_strings:
            f.write(f"{string}\n")
    print(f"Successfully saved {len(pauli_strings)} Pauli strings to '{strings_filename}'.")

    # --- Sequential Execution on GPU ---
    print(f"\nStarting computation for {len(pauli_strings)} matrices on the GPU...")
    start_time = time.time()

    for pauli_string in pauli_strings:
        gpu_matrix = gpu_hamiltonian_generator.tensor_product(pauli_string)
        cpu_matrix = gpu_matrix.get()
        filename = os.path.join(OUTPUT_DIR, f"{pauli_string}.npz")
        save_npz(filename, cpu_matrix)

    end_time = time.time()
    print(f"\nProcessing complete. Saved {len(pauli_strings)} matrix files in {end_time - start_time:.2f} seconds.")

GPU Device 0 detected. Running on GPU.
Matrices will be saved in the 'gpu_pauli_matrices/' directory.
Starting computation for 10 matrices on the GPU...

Processing complete. Saved 10 matrix files in 0.64 seconds.
Example filenames: ['ZIYZIIYZIYIIXII.npz', 'YIIIIIZIZIYYXYI.npz', 'YIYIIIXYIXIIZXI.npz', 'IIYXIYZXIXIIIYI.npz', 'YIXIZIIIIIIXZXZ.npz']


In [ ]:
import cupy as cp
import cupyx.scipy.sparse as cupy_sparse
import numpy as np
from itertools import combinations, product
from functools import reduce
from scipy.sparse import save_npz # Keep for saving the final result on the CPU
import math
import os
import time

class GPUKLocalHamiltonian:
    """
    Constructs and processes k-local Pauli strings on the GPU using CuPy.
    """
    def __init__(self, L, k, adjacent=True):
        self.L = L
        self.k = k
        self.adjacent = adjacent
        self.paulis = ['X', 'Y', 'Z']
        # String generation is a lightweight CPU task
        self.k_local_paulis = self.generate_k_local_pauli_strings()

    def pauli_matrix(self, p):
        """Returns the sparse CuPy matrix for a single Pauli operator."""
        if p == "I":
            return cupy_sparse.identity(2, format="csr", dtype=cp.complex128)
        elif p == "X":
            return cupy_sparse.csr_matrix(cp.array([[0, 1], [1, 0]], dtype=cp.complex128))
        elif p == "Y":
            return cupy_sparse.csr_matrix(cp.array([[0, -1j], [1j, 0]], dtype=cp.complex128))
        elif p == "Z":
            return cupy_sparse.csr_matrix(cp.array([[1, 0], [0, -1]], dtype=cp.complex128))
        else:
            raise ValueError(f"Invalid Pauli operator: {p}")

    def tensor_product(self, pauli_string):
        """Computes the tensor product for a full Pauli string on the GPU."""
        matrices = [self.pauli_matrix(p) for p in pauli_string]
        return reduce(cupy_sparse.kron, matrices)

    def generate_k_local_pauli_strings(self):
        """Generates all k-local Pauli strings on the CPU."""
        pauli_strings = []
        if self.adjacent:
            for i in range(self.L - self.k + 1):
                for active_ops in product(self.paulis, repeat=self.k):
                    pauli_string = ['I'] * self.L
                    for offset, P in enumerate(active_ops):
                        pauli_string[i + offset] = P
                    pauli_strings.append("".join(pauli_string))
        else: # Non-adjacent
            for indices in combinations(range(self.L), self.k):
                for active_ops in product(self.paulis, repeat=self.k):
                    pauli_string = ['I'] * self.L
                    for idx, P in zip(indices, active_ops):
                        pauli_string[idx] = P
                    pauli_strings.append("".join(pauli_string))
        return pauli_strings

# --- Main execution block ---
if __name__ == "__main__":
    # --- GPU Check ---
    try:
        device_id = cp.cuda.runtime.getDevice()
        print(f"✅ GPU Device {device_id} detected. Running on GPU.")
    except cp.cuda.runtime.CUDARuntimeError as e:
        print(f"GPU not detected or CuPy installation issue. Error: {e}")
        exit()

    # --- 1. Set Parameters ---
    adjacent = True
    # NOTE: Using smaller L and k for a quick demonstration.
    L = 17
    k = 7
    OUTPUT_DIR = "pauli_string_matrices"
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    print(f"Setting up for L={L}, k={k}, adjacent={adjacent}")
    print(f"Matrices will be saved in '{OUTPUT_DIR}/'")

    # --- 2. Instantiate the Builder ---
    gpu_hamiltonian_builder = GPUKLocalHamiltonian(L=L, k=k, adjacent=adjacent)
    pauli_strings = gpu_hamiltonian_builder.k_local_paulis
    print(f"Generated {len(pauli_strings)} Pauli strings.")

    # --- 3. Compute and Save Each Matrix ---
    print("\nProcessing and saving each Pauli string matrix...")
    start_time = time.time()

    for pauli_string in pauli_strings:
        # Compute the tensor product on the GPU
        gpu_matrix = gpu_hamiltonian_builder.tensor_product(pauli_string)

        # Move the result to the CPU for saving
        cpu_matrix = gpu_matrix.get()

        # Save the matrix with its string as the filename
        filename = os.path.join(OUTPUT_DIR, f"{pauli_string}.npz")
        save_npz(filename, cpu_matrix)

    cp.cuda.Stream.null.synchronize() # Wait for all GPU operations to finish
    end_time = time.time()

    print(f"\nProcessing complete. Saved {len(pauli_strings)} matrix files in {end_time - start_time:.2f} seconds.")
    print("Example filenames:", os.listdir(OUTPUT_DIR)[:5])

✅ GPU Device 0 detected. Running on GPU.
Setting up for L=17, k=7, adjacent=True
Matrices will be saved in 'pauli_string_matrices/'
Generated 24057 Pauli strings.

Processing and saving each Pauli string matrix...

Processing complete. Saved 24057 matrix files in 4393.98 seconds.
Example filenames: ['IIIIIIIIZYYYXYXII.npz', 'IIIIIIIIIYXXYZYZI.npz', 'IIIIIYZXZZXZIIIII.npz', 'IXZZZZZYIIIIIIIII.npz', 'IIYZZYYXZIIIIIIII.npz']


In [ ]:
!zip -r /content/pauli_string_matrices.zip /content/pauli_string_matrices
# from google.colab import files
# files.download('/content/pauli_string_matrices.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>